In [1]:
import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt
import csv
import math

# ビデオデータのパス
video_path = "C:\\Users\\flow\\Desktop\\bubbledate\\00089.mp4"

# 動画の読み込み
cap = cv2.VideoCapture(filename=video_path)

# 動画ファイルの情報を取得
video_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
video_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)

# 動画情報を表示
print("動画の幅: ", video_width)
print("動画の高さ: ", video_height)
print("総フレーム数: ", total_frames)
print("fps: ", fps)
print("動画の長さ: ", total_frames / fps, "[sec]")


動画の幅:  1920.0
動画の高さ:  1080.0
総フレーム数:  3449.0
fps:  29.97002997002997
動画の長さ:  115.08163333333333 [sec]


In [7]:
import os

# ルートフォルダーのパス
root_path = "C:\\Users\\flow\\Desktop\\bubble_rasing_result"

# サブフォルダーの名前
image_folder = "image"
area_folder = "area"
contours_folder = "contours"
moment_folder = "moment"

# フォルダーのパスを連結
path = os.path.join(root_path, image_folder)
path1 = os.path.join(root_path, area_folder)
path2 = os.path.join(root_path, contours_folder)
path3 = os.path.join(root_path, moment_folder)

print(path,path1,path2,path3,path)

# パスとして使用する画像番号の桁数（3桁として扱う）
image_number_digits = 3

# 2値化の閾値として使用する値（この値を調整して結果を確認できます）
threshold_value = 70

# 輪郭のインデックス（面積順にソートされるため、0が最大の面積の輪郭になる）
contour_index = 2

C:\Users\flow\Desktop\bubble_rasing_result\image C:\Users\flow\Desktop\bubble_rasing_result\area C:\Users\flow\Desktop\bubble_rasing_result\contours C:\Users\flow\Desktop\bubble_rasing_result\moment C:\Users\flow\Desktop\bubble_rasing_result\image


In [8]:
def process_image(frame, num, path, path1, path3, cx_list, cy_list, area_list, number_list):
    # 画像を保存
    image_path = os.path.join(path, f"picture{num:0={image_number_digits}}.bmp")
    cv2.imwrite(image_path, frame)

    # 画像を読み込み
    img1 = cv2.imread(image_path)
    # グレースケールに変換
    gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY) 
    # 平滑化
    blur = cv2.bilateralFilter(gray, 5, 10, 10)
    # グレースケール画像を保存
    gray_path = os.path.join(path, f"graypicture{num:0={image_number_digits}}.bmp")
    cv2.imwrite(gray_path, blur)

    # 2値化する
    ret, binary = cv2.threshold(blur, threshold_value, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    binary_path = os.path.join(path, f"nichipicture{num:0={image_number_digits}}.bmp")
    cv2.imwrite(binary_path, binary)

    # 輪郭を検出
    contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    contours.sort(key=lambda x: cv2.contourArea(x), reverse=True)

    # 輪郭を描画
    img2 = cv2.imread(image_path)
    cv2.drawContours(img2, [contours[contour_index]], -1, (0, 255, 0), 2)
    rinkaku_path = os.path.join(path1, f"rinkakupicture{num:0={image_number_digits}}.bmp")
    cv2.imwrite(rinkaku_path, img2)

    # 輪郭の情報を取得
    cnt = contours[contour_index]
    area = cv2.contourArea(cnt)
    M = cv2.moments(cnt)
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])

    # 重心を描画
    moment = cv2.imread(image_path)
    cv2.circle(moment, (cx, cy), 8, (0, 0, 255), -1)

    # 輪郭を描画
    contour_img = cv2.imread(image_path)
    cv2.drawContours(contour_img, [cnt], -1, (0, 255, 0), 2)

    # 重心と輪郭を結合して保存
    combined_img = cv2.addWeighted(moment, 0.5, contour_img, 0.5, 0)

    # 保存するファイルパス
    combined_img_path = os.path.join(path3, f"moment_with_contour{num:0={image_number_digits}}.bmp")
    cv2.imwrite(combined_img_path, combined_img)

    # リストに追加
    cx_list.append(cx)
    cy_list.append(cy)
    area_list.append(area)
    number_list.append(num)

    return cx_list, cy_list, area_list, number_list